Descriptions:

# 1) Functions & Classes

In [ ]:
import math
import numpy as np
from scipy.spatial import ConvexHull
import time
import numpy as np
import matplotlib.pyplot as plt

## 1.1 PGM-Index subroutines

In [ ]:
def minimum_bounding_rectangle(points):
   ### Ref: https://stackoverflow.com/questions/13542855/algorithm-to-find-the-minimum-area-rectangle-for-given-points-in-order-to-comput
    """
    Find the smallest bounding rectangle for a set of points.
    Returns a set of points representing the corners of the bounding box.

    :param points: an nx2 matrix of coordinates
    :rval: an nx2 matrix of coordinates
    """
    from scipy.ndimage.interpolation import rotate
    pi2 = np.pi/2.

    # get the convex hull for the points
    try:
      hull_points = points[ConvexHull(points).vertices]
    except:
      return -2


    # calculate edge angles
    edges = np.zeros((len(hull_points)-1, 2))
    edges = hull_points[1:] - hull_points[:-1]

    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])

    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)

    # find rotation matrices
    # XXX both work
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles-pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T

    rotations = rotations.reshape((-1, 2, 2))

    # apply rotations to the hull
    rot_points = np.dot(rotations, hull_points.T)

    # find the bounding points
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)

    # find the box with the best area
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)

    # return the best box
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]
    r = rotations[best_idx]

    rval = np.zeros((4, 2))
    rval[0] = np.dot([x1, y2], r)
    rval[1] = np.dot([x2, y2], r)
    rval[2] = np.dot([x2, y1], r)
    rval[3] = np.dot([x1, y1], r)

    return rval

In [ ]:
import numpy as np
def generate_keys(size = 10**6, max = 10**6, decimals = 3):
  """
  size: total number of keys generated
  max: maximum value of a key. The range is [0,max)
  decimal: number of decimal places of keys. Keys are float numbers
  we assume that positions start from 0 up to size-1 for the sorted keys
  """
  # we generate 10% more keys since at the end we only need unique keys
  n = size + int(size * 0.1)
  x = np.random.random((n,1))
  x = x.dot(max)
  x = np.around(x, decimals=decimals)
  x = np.sort(x, axis=0)
  x = np.unique(x, axis=0)
  assert x.shape[0] >= size , \
         f"{x.shape=}, we cannot slice {size=} from it"
  x = x[:size, 0]
  positions = list(range(size))
  return x , positions


In [ ]:
def reorder_points(points):
  """
  sort points in the order that
  1st element: with the lowest y value
  2nd: highest y value
  3rd: lowest x value
  4th: highest x value
  """
  result = np.array(points)
  x_val, y_val = points[0][0], points[0][1]
  indices = list(range(points.shape[0]))
  temp_index = -1
  for i in indices:
    if(points[i][1] <= y_val):
      temp_index = i
      y_val = points[i][1]
  #found 1st. Remove its index from the list & store it at the right place
  indices.remove(temp_index)
  result[0] = points[temp_index]

  for i in indices:
    if(points[i][1] >= y_val):
      temp_index = i
      y_val = points[i][1]
  #found 2nd.
  indices.remove(temp_index)
  result[1] = points[temp_index]

  for i in indices:
    if(points[i][0] <= x_val):
      temp_index = i
      x_val = points[i][0]
  #found 3rd.
  indices.remove(temp_index)
  result[2] = points[temp_index]

  # the last element's index is left in indices array
  result[3] = points[indices[0]]

  return result

In [ ]:
def collinear(keys, new_p):
  # we already know the y_value for the segement are increasing
  # one by one per element so we don't need it for checking slopes
  return True if ((keys[0]-keys[1])==(keys[-1]-new_p)) else False

In [ ]:
def get_slope(x1, y1, x2, y2):
  slope = (y2-y1)/(x2-x1)
  return slope

def get_Y_ic(x1, y1, x2, y2):
  """ y=mx+b >> b=y-mx """
  s = get_slope(x1, y1, x2, y2)
  b=y1-s*x1
  return b

In [ ]:
def height_finder(keys, indices):
  # call convex hull then bbox then reorder then calculate h here
  h = -1
  k = np.array(keys)
  p = np.array(indices)
  points = np.column_stack((k,p))
  bbox = minimum_bounding_rectangle(points)
  if (type(bbox) == type(-2)):
    return 0 # height is zero if points are almost colinear
  points = reorder_points(bbox)

  x1, y1= points[1,:]
  x2, y2= points[2,:]
  slope_2nd_3rd = get_slope(x1, y1, x2, y2)
  ic_2nd_3rd = get_Y_ic(x1, y1, x2, y2)
  # print(f"{x1=}, {y1=}, {x2=}, {y2=}")
  # print(f"{slope_2nd_3rd=}, and {ic_2nd_3rd=}")
  h = (slope_2nd_3rd * points[0,0] + ic_2nd_3rd) - points[0,1]
  assert h >= 0 ,  f"Invalid height{h=}"

  return h

In [ ]:
def extract_first_sl_ic_for2points(keys, indices):
  sl = get_slope(keys[0], indices[0], keys[1], indices[1])
  ic = get_Y_ic(keys[0], indices[0], keys[1], indices[1])
  return (keys[0], sl , ic)


In [ ]:
def extract_first_sl_ic(keys, indices):
  k = np.array(keys)
  p = np.array(indices)
  points = np.column_stack((k,p))
  bbox = minimum_bounding_rectangle(points)
  if type(bbox) == type(-2):
    return extract_first_sl_ic_for2points(keys[:2], indices[:2])
  points = reorder_points(bbox)
  x1, y1= (points[0,0]+ points[2,0])/2 , (points[0,1]+ points[2,1])/2
  x2, y2= (points[1,0]+ points[3,0])/2 , (points[1,1]+ points[3,1])/2
  slope = get_slope(x1, y1, x2, y2)
  ic = get_Y_ic(x1, y1, x2, y2)

  return (keys[0], slope, ic)

## 1.2 PGM-*Index

In [ ]:
def build_PLA_model(keys,n,eps,pos):
  PLA_models = []
  segment_keys = [] # current segment points
  indices = []
  linear = False
  tolerance = 0.5
  eps -= tolerance
  """
  On adding a new segment to PLA_models
  + we flush the segment_keys and add the point that was propabebly
  causing over height.
  + reset linear to False
  """
  for i in range(n):
    #// print(f"{segment_keys=}, {indices=}")
    if len(segment_keys) <= 1:
      segment_keys.append(keys[i])
      indices.append(pos[i])
    else:
      # now we have already two points. Gonna add the third
      if (linear==True ) or (len(segment_keys) == 2):
        # we already know the y_value for the segement are increasing
        # one by one per element so we don't need it
        linear = collinear(segment_keys,keys[i])

      if linear:
        segment_keys.append(keys[i])
        indices.append(pos[i])
      else:
        # we temporarly add keys[i] and see if height is still <= 2e
        # here we have atleast 3 points & are not collinear
        h = height_finder(segment_keys + [keys[i]], indices + [pos[i]])
        #// print(f'{h=} and {eps=}')
        if h > 2*eps:
          if len(segment_keys) == 2:
            PLA_models.append(extract_first_sl_ic_for2points(segment_keys, indices))
          else:
            PLA_models.append(extract_first_sl_ic(segment_keys, indices))
          segment_keys = [] # reset
          indices = [] # reset
          linear = False # reset
          segment_keys.append(keys[i])
          indices.append(pos[i])
        else:
          segment_keys.append(keys[i])
          indices.append(pos[i])

  # At the end of "for" we check if anything is left in segment_keys
  # Three possible scenarios exist
  # (1) only one point is left! (2) two points, (3) more that two points
  # we only need to add the tripple (first_key, slop, ic) describing
  # remained segment
  rem = len(segment_keys)
  if rem == 0:
    pass
  elif rem == 1:
    # we suppose we had a previous point (0,0)
    sl = get_slope(segment_keys[0], indices[0], 0, 0)
    ic = get_Y_ic(segment_keys[0], indices[0], 0,0)
    PLA_models.append((segment_keys[0], sl , ic))
  elif rem == 2:
    PLA_models.append(extract_first_sl_ic_for2points(segment_keys, indices))
  else:
    PLA_models.append(extract_first_sl_ic(segment_keys, indices))

  return PLA_models

In [ ]:
def build_pgm_index(A, n, eps =2, pos=[]):
  """
  A: list of keys. They are X values.
  n: len(A)
  pos: list of positions of keys. These are Y values.
  eps: PGM model's parameter
  """
  if pos == []:
    pos = list(range(n))
  levels = []
  keys = A

  while(n != 1):
    M = build_PLA_model(keys,n,eps,pos)
    levels.append(M)
    n = len(M)
    keys = [tripple[0] for tripple in M ]
    pos = list(range(n))

  levels.reverse()
  return levels

In [ ]:
def right_most_lowest_segment(level, key_query , low, high):
  """
  Returns a the if of the right segment whose key is < key_query
  from low index up to high, inclusive.
  level[i] = (key, slope, intercept)
  """
  id = -1
  for i in range(high, low-1, -1):
    if(level[i][0] <= key_query):
      return i

  print("inside the right most func")
  print(level)
  print(key_query)
  print(low)
  print(high)
  assert id > -1

In [ ]:
def find_pos_for_nex_level(id, level, k):
  size = len(level)
  #print(f"{id=}, {level=}, {k=}")
  s = level[id]
  if id == (size -1):
    return math.floor(k * s[1]+ s[2])
  return math.floor(min(k * s[1]+ s[2], level[id+1][0]*level[id+1][1]+level[id+1][2]))


In [ ]:
def find_a_key(A, key_query , low, high):
  index = -1
  for i in range(low, high+1):
    if A[i] == key_query:
      index = i
      break
  return index


In [ ]:
def search(A, n, eps, levels, key_query):
  """
  This function gets a key as a query and
  return its index of that key in original Array
  if not found returns -1
  """
  idx = -1
  if key_query < levels[0][0][0]:
    return -1
  pos = key_query * levels[0][0][1] + levels[0][0][2]
  pos = round(pos)

  for i in range(1,len(levels)):
    low = max(0, pos - eps)
    high = min( len(levels[i])-1, pos + eps)
    id = right_most_lowest_segment(levels[i], key_query , low, high)
    pos = find_pos_for_nex_level(id, levels[i], key_query)
    #print(f"inside {low=} and {id=}, and {high=}, {pos=}")
  low = max(0, pos - eps)
  high = min(len(A)-1, pos + eps)
  #print(f"final {low=} and {high=}, {pos=}")
  # now by this low and high we search for the key in A.
  # we can reuse the right_most_lowest_segment func
  target_id = find_a_key(A, key_query , low, high)
  return target_id


## 1.3 B-plus tree

In [ ]:
# Python3 program for implementing B+ Tree

import math

# Node creation
class Node:
	def __init__(self, order):
		self.order = order
		self.values = []
		self.keys = []
		self.nextKey = None
		self.parent = None
		self.check_leaf = False

	# Insert at the leaf
	def insert_at_leaf(self, leaf, value, key):
		if (self.values):
			temp1 = self.values
			for i in range(len(temp1)):
				if (value == temp1[i]):
					self.keys[i].append(key)
					break
				elif (value < temp1[i]):
					self.values = self.values[:i] + [value] + self.values[i:]
					self.keys = self.keys[:i] + [[key]] + self.keys[i:]
					break
				elif (i + 1 == len(temp1)):
					self.values.append(value)
					self.keys.append([key])
					break
		else:
			self.values = [value]
			self.keys = [[key]]


# B plus tree
class BplusTree:
	def __init__(self, order):
		self.root = Node(order)
		self.root.check_leaf = True

	# Insert operation
	def insert(self, value, key):
		value = str(value)
		old_node = self.search(value)
		old_node.insert_at_leaf(old_node, value, key)

		if (len(old_node.values) == old_node.order):
			node1 = Node(old_node.order)
			node1.check_leaf = True
			node1.parent = old_node.parent
			mid = int(math.ceil(old_node.order / 2)) - 1
			node1.values = old_node.values[mid + 1:]
			node1.keys = old_node.keys[mid + 1:]
			node1.nextKey = old_node.nextKey
			old_node.values = old_node.values[:mid + 1]
			old_node.keys = old_node.keys[:mid + 1]
			old_node.nextKey = node1
			self.insert_in_parent(old_node, node1.values[0], node1)

	# Search operation for different operations
	def search(self, value):
		current_node = self.root
		while(current_node.check_leaf == False):
			temp2 = current_node.values
			for i in range(len(temp2)):
				if (value == temp2[i]):
					current_node = current_node.keys[i + 1]
					break
				elif (value < temp2[i]):
					current_node = current_node.keys[i]
					break
				elif (i + 1 == len(current_node.values)):
					current_node = current_node.keys[i + 1]
					break
		return current_node

	# Find the node
	def find(self, value, key):
		l = self.search(value)
		for i, item in enumerate(l.values):
			if item == value:
				if key in l.keys[i]:
					return True
				else:
					return False
		return False

	# Inserting at the parent
	def insert_in_parent(self, n, value, ndash):
		if (self.root == n):
			rootNode = Node(n.order)
			rootNode.values = [value]
			rootNode.keys = [n, ndash]
			self.root = rootNode
			n.parent = rootNode
			ndash.parent = rootNode
			return

		parentNode = n.parent
		temp3 = parentNode.keys
		for i in range(len(temp3)):
			if (temp3[i] == n):
				parentNode.values = parentNode.values[:i] + \
					[value] + parentNode.values[i:]
				parentNode.keys = parentNode.keys[:i +
												1] + [ndash] + parentNode.keys[i + 1:]
				if (len(parentNode.keys) > parentNode.order):
					parentdash = Node(parentNode.order)
					parentdash.parent = parentNode.parent
					mid = int(math.ceil(parentNode.order / 2)) - 1
					parentdash.values = parentNode.values[mid + 1:]
					parentdash.keys = parentNode.keys[mid + 1:]
					value_ = parentNode.values[mid]
					if (mid == 0):
						parentNode.values = parentNode.values[:mid + 1]
					else:
						parentNode.values = parentNode.values[:mid]
					parentNode.keys = parentNode.keys[:mid + 1]
					for j in parentNode.keys:
						j.parent = parentNode
					for j in parentdash.keys:
						j.parent = parentdash
					self.insert_in_parent(parentNode, value_, parentdash)

# Print the tree
def printTree(tree):
	lst = [tree.root]
	level = [0]
	leaf = None
	flag = 0
	lev_leaf = 0

	node1 = Node(str(level[0]) + str(tree.root.values))

	while (len(lst) != 0):
		x = lst.pop(0)
		lev = level.pop(0)
		if (x.check_leaf == False):
			for i, item in enumerate(x.keys):
				print(item.values)
		else:
			for i, item in enumerate(x.keys):
				print(item.values)
			if (flag == 0):
				lev_leaf = lev
				leaf = x
				flag = 1

# 2) Driver Code

In [ ]:
## Generating data
##################### ******************* ###########################
keys, positions = generate_keys(size = 10**6, max = 10**5, decimals = 3)
k = keys.tolist()

In [ ]:
## Testing PGM model
for j in range(2,11):
  print(f"Before processing {j=}")
  model = build_pgm_index(k, len(k), eps =j, pos=positions)
  for i in range(len(k)):
    if i != search(k, len(k), j, model, k[i]):
      print(i)

  print(f"finished processing {j=}")

Before processing j=2


<ipython-input-2-03f8b29d70b5>:10: DeprecationWarning: Please use `rotate` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import rotate


finished processing j=2
Before processing j=3
finished processing j=3
Before processing j=4
finished processing j=4
Before processing j=5
finished processing j=5
Before processing j=6
finished processing j=6
Before processing j=7
finished processing j=7
Before processing j=8
finished processing j=8
Before processing j=9
finished processing j=9
Before processing j=10
finished processing j=10


In [ ]:
### B-plus tree
record_len = 3
bplustree = BplusTree(record_len)


## Inserting Keys
for i in range(len(k)):
  # insert(value, key)
  bplustree.insert(str(k[i]), str(i))

## performing search
for i in range(len(k)):
  # insert(value, key)
  if(not bplustree.find(str(k[i]), str(i))):
    print(i)

In [ ]:
#Testing B-plus tree (Note for J =2 we reach Maximum recurssion depth.)
for j in range(3,11):
  print(f"Before processing {j=}")
  record_len = j
  bplustree = BplusTree(record_len)
  ## Inserting Keys
  for i in range(len(k)):
    # insert(value, key)
    bplustree.insert(str(k[i]), str(i))

  ## performing search
  for i in range(len(k)):
    # insert(value, key)
    if(not bplustree.find(str(k[i]), str(i))):
      print(i)
  print(f"after processing {j=}")

Before processing j=3
after processing j=3
Before processing j=4
after processing j=4
Before processing j=5
after processing j=5
Before processing j=6
after processing j=6
Before processing j=7
after processing j=7
Before processing j=8
after processing j=8
Before processing j=9
after processing j=9
Before processing j=10
after processing j=10


# 10) References